Tensor RTに変換された学習済みモデルをつかって自動走行します。

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Tensor RT形式のモデルを読み込む。

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

racecarクラスをインスタンス化する

In [3]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

96


カメラの起動のためにカメラを制御するnvargus-daemonの再起動。

In [4]:
!echo jetson | sudo -S systemctl restart nvargus-daemon

[sudo] password for jetson: 

カメラクラスをインスタンス化する。

In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=40)

最後に、JetRacerを下に置き、下記セルを実行します。

* 車の左右にうまく回らない場合は、`STEERING_GAIN` を小さいくする
* ターンがうまくいかない場合は、`STEERING_GAIN`を大きくする
* 車が左に傾く場合は、`STEERING_BIAS`を値を-0.05ぐらいづつマイナスにする
* 車が右に傾く場合は、`STEERING_BIAS`を値を+0.05ぐらいづつプラスにする

In [6]:
import ipywidgets.widgets as widgets
from IPython.display import display
from utils import preprocess
import numpy as np
import threading
import traitlets

throttle_slider = widgets.FloatSlider(description='throttle', min=-1.0, max=1.0, step=0.01, value=0.15, orientation='vertical')
steering_gain = widgets.BoundedFloatText(description='st_gain',min=-1.0, max=1.0, step=0.01, value=-0.65)
steering_offset = widgets.BoundedFloatText(description='st_offset',min=-1.0, max=1.0, step=0.01, value=0.2)
run_button = widgets.Button(description='推論開始')
stop_button = widgets.Button(description='推論停止')
log_widget = widgets.Textarea(description='ログ')
result_widget = widgets.FloatText(description='推論から導いたXの値')

def live():
    global running
    log_widget.value = "live"
    i = 0
    while running:
        i = i + 1
        log_widget.value = str(i) + "回目の推論"
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        steering_value = x * car.steering_gain + car.steering_offset
        result_widget.value = steering_value
        car.steering = steering_value

def run(c):
    global running, execute_thread
    log_widget.value = "run"
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()

def stop(c):
    global running, execute_thread
    log_widget.value = "stop"
    running = False
    execute_thread.stop()
        
run_button.on_click(run)
stop_button.on_click(stop)

# create a horizontal box container to place the sliders next to eachother
run_widget = widgets.VBox([
    widgets.HBox([throttle_slider, steering_gain,steering_offset]),
    widgets.HBox([run_button, stop_button]),
    result_widget,
    log_widget
])
throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))

# display the container in this cell's output
display(run_widget)